In [2]:
#!pip install langchain langchain-community langchain_groq chromadb gradio

In [3]:
import pandas as pd
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders.csv_loader import CSVLoader

In [4]:
from langchain_groq import ChatGroq

In [5]:
data = pd.read_csv("/content/drive/MyDrive/Anime Movie Recommender/anime_with_synopsis.csv")
data.head()

,MAL_ID,Name,Score,Genres,sypnopsis
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space","In the year 2071, humanity has colonized sever..."
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space","other day, another bounty—such is the life of ..."
2,6,Trigun,8.24,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen","Vash the Stampede is the man with a $$60,000,0..."
3,7,Witch Hunter Robin,7.27,"Action, Mystery, Police, Supernatural, Drama, ...",ches are individuals with special powers like ...
4,8,Bouken Ou Beet,6.98,"Adventure, Fantasy, Shounen, Supernatural",It is the dark century and the people are suff...


In [6]:
data.isnull().sum()

,0
MAL_ID,0
Name,0
Score,0
Genres,0
sypnopsis,0


#### data may have some NAN values as it is textual data so remove them

In [7]:
data.dropna()

,MAL_ID,Name,Score,Genres,sypnopsis
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space","In the year 2071, humanity has colonized sever..."
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space","other day, another bounty—such is the life of ..."
2,6,Trigun,8.24,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen","Vash the Stampede is the man with a $$60,000,0..."
3,7,Witch Hunter Robin,7.27,"Action, Mystery, Police, Supernatural, Drama, ...",ches are individuals with special powers like ...
4,8,Bouken Ou Beet,6.98,"Adventure, Fantasy, Shounen, Supernatural",It is the dark century and the people are suff...
...,...,...,...,...,...
264,296,Dragon Drive,6.73,"Action, Sci-Fi, Adventure, Comedy, Fantasy, Sc...","If there's one word to describe Reiji Ozora, i..."
265,297,Grenadier: Hohoemi no Senshi,6.72,"Action, Ecchi, Adventure, Comedy, Shounen",ushuna is a blonde and very beautiful Senshi (...
266,298,.hack//Tasogare no Udewa Densetsu,6.60,"Adventure, Comedy, Fantasy, Game, Sci-Fi, Shounen","g the legendary characters ""Kite"" and ""Black R..."
267,299,.hack//Liminality,6.61,"Game, Mystery, Sci-Fi","hile playing the newly released MMORPG ""The Wo..."


#### Now will combine both genres and sypnopsis column to find patterns
- we are creating a new summary coulmn for each anime by combining information from its name, description, and genres into a single, easy-to-read sentence

In [8]:
data['combined_df'] =data.apply(lambda row: f"Title: {row['Name']}. Overview : {row['sypnopsis']} Genres : {row['Genres']}", axis = 1)


In [9]:
data['combined_df'][2]

'Title: Trigun. Overview : Vash the Stampede is the man with a $$60,000,000,000 bounty on his head. The reason: he\'s a merciless villain who lays waste to all those that oppose him and flattens entire cities for fun, garnering him the title "The Humanoid Typhoon." He leaves a trail of death and destruction wherever he goes, and anyone can count themselves dead if they so much as make eye contact—or so the rumors say. In actuality, Vash is a huge softie who claims to have never taken a life and avoids violence at all costs. With his crazy doughnut obsession and buffoonish attitude in tow, Vash traverses the wasteland of the planet Gunsmoke, all the while followed by two insurance agents, Meryl Stryfe and Milly Thompson, who attempt to minimize his impact on the public. But soon, their misadventures evolve into life-or-death situations as a group of legendary assassins are summoned to bring about suffering to the trio. Vash\'s agonizing past will be unraveled and his morality and princi

In [10]:
data[['combined_df']].to_csv('updated_anime.csv', index=False)

In [11]:
pd.read_csv('/content/updated_anime.csv')

,combined_df
0,Title: Cowboy Bebop. Overview : In the year 20...
1,Title: Cowboy Bebop: Tengoku no Tobira. Overvi...
2,Title: Trigun. Overview : Vash the Stampede is...
3,Title: Witch Hunter Robin. Overview : ches are...
4,Title: Bouken Ou Beet. Overview : It is the da...
...,...
264,Title: Dragon Drive. Overview : If there's one...
265,Title: Grenadier: Hohoemi no Senshi. Overview ...
266,Title: .hack//Tasogare no Udewa Densetsu. Over...
267,Title: .hack//Liminality. Overview : hile play...


In [ ]:
loader = CSVLoader(file_path='/content/updated_anime.csv')
new_data = loader.load()

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(new_data)

embeddings = HuggingFaceEmbeddings()
llm = ChatGroq(api_key = "your api key", model = "llama-3.3-70b-versatile", temperature= 0)

docssearch = Chroma.from_documents(texts, embeddings)

<ipython-input-12-0bfadf0ece92>:7: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings()
<ipython-input-12-0bfadf0ece92>:7: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggin

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## Summary of above Code:
1. Load anime data from a CSV file.
2. Split the data into manageable pieces.
3. Create embeddings for those pieces to represent their meaning.
4. Set up a language model for generating text or answering questions.
5. Create a searchable database based on the text chunks and their embeddings.



**Querying vector DB for movie recommendation**

In [13]:
query = "I am looking for an war animated movie"
docs = docssearch.similarity_search(query)

In [14]:
docs

[Document(metadata={'row': 234, 'source': '/content/updated_anime.csv'}, page_content="combined_df: Title: Gunparade March: Arata Naru Kougunka. Overview : It began in 1945, at the end of the Pacific War. Alien invaders filled the earth's Skies, and mankind was forced to confront an unprecedented threat. For the first time in human history, people of all cultures came together under one banner. This war has now been raging for over fifty years. Countless lives have been lost, and the Japanese military is now forced to rely on young people such as Atsushi Hayami and his high school class, also known as Unit 5121. This new generation fearlessly struggles on with the aid of the HWT humanoid combat machines and the devastating PBE bomb. (Source: Media Blasters) Genres : Drama, Mecha, Military, Romance, Sci-Fi, Shounen, Slice of Life"),
 Document(metadata={'row': 19, 'source': '/content/updated_anime.csv'}, page_content="combined_df: Title: Zipang. Overview : ai, an improved Kongou-class Ae

**Now QA Retrieval System for Movie Recommendation**

In [15]:
qa = RetrievalQA.from_chain_type(llm,
            chain_type = "stuff",
            retriever = docssearch.as_retriever(),
            return_source_documents = True)

## Summary of above code
1. **Retrieval**: The system uses the retriever to find relevant documents from the previously processed anime data.
2. **Response Generation**: The language model (llm) then takes these documents and formulates an answer based on the content of those documents.
3. **Source Retrieval**: The response will include references to the original documents, making it clear where the information came from.

In [16]:
query = "Can you recommend a anime with a unique storyline?"
result = qa({"query": query})


<ipython-input-16-d1e762d86cb8>:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa({"query": query})


In [17]:
result['result']

'Based on the provided context, I would recommend "RahXephon" or "Loveless" for their unique storylines. \n\n"RahXephon" has a complex plot involving a time bubble, humanoids from another dimension, and a protagonist who discovers a powerful being that changes his life. \n\n"Loveless" has a distinctive world where people are born with cat ears and tails that disappear after they engage in sexual intercourse, and it explores themes of virginity, innocence, and fighting pairs.\n\nBoth of these anime series have intriguing and unconventional storylines that might interest you.'

In [18]:
result['source_documents'][0]

Document(metadata={'row': 204, 'source': '/content/updated_anime.csv'}, page_content="combined_df: Title: E's Otherwise. Overview : Kai, who has powers different from the rest, together with his sickly sister Hikaru was protected by an organization called ASHURUM. Scouted by Eiji, Kai was delegated to the ASHURUM special force AESES and undergo intensive training. Whenever he was free, Kai visited Hikaru at the hospital belongs to the organization, but Hikaru's condition never improved. So, one year later, with amazing growth from the intensive training, Kai decides to escape from the organization. (Source: ANN) Genres : Adventure, Comedy, Drama, Military, Sci-Fi, Shounen, Supernatural")

In [19]:
from langchain.prompts import PromptTemplate

template = """You are a movie recommender system that help users to find anime that match their preferences.
Use the following pieces of context to answer the question at the end.
For each question, suggest three anime, with a short description of the plot and the reason why the user migth like it.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Your response:"""

PROMPT = PromptTemplate(
    template=template,
    input_variables=['context', 'question']
)

qa = RetrievalQA.from_chain_type(llm,
            chain_type = "stuff",
            retriever = docssearch.as_retriever(),
            return_source_documents = True)
query = "Give me a list of war animes that are lighthearted and fun."
result = qa({"query": query})
print(result['result'])

Based on the provided context, here are some war anime recommendations that may be considered lighthearted and fun:

1. Grenadier: Hohoemi no Senshi - This anime has a mix of action, comedy, and adventure, and its main character's goal is to make the world a peaceful place by giving people a smile, which suggests a more lighthearted tone.

The other anime listed in the context, such as Gunparade March: Arata Naru Kougunka, Seikai no Monshou, and Area 88 (TV), seem to have more serious and dramatic storylines, and may not fit the "lighthearted and fun" category.

Please note that the definition of "lighthearted and fun" can vary depending on personal taste, and it's always a good idea to read more about an anime before watching it to ensure it aligns with your expectations.


#### 2nd Prompt

In [20]:
from langchain.prompts import PromptTemplate

template_prefix = """You are a movie recommender system that help users to find anime that match their preferences.
Use the following pieces of context to answer the question at the end.
For each question, take into account the context and the personal information provided by the user.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}"""

user_info = """This is what we know about the user, and you can use this information to better tune your research:
Age: {age}
Gender: {gender}"""

template_suffix= """Question: {question}
Your response:"""

user_info = user_info.format(age = 18, gender = 'male')

COMBINED_PROMPT = template_prefix +'\n'+ user_info +'\n'+ template_suffix
print(COMBINED_PROMPT)

You are a movie recommender system that help users to find anime that match their preferences.
Use the following pieces of context to answer the question at the end.
For each question, take into account the context and the personal information provided by the user.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}
This is what we know about the user, and you can use this information to better tune your research:
Age: 18
Gender: male
Question: {question}
Your response:


In [21]:
PROMPT  =  PromptTemplate(
    template=COMBINED_PROMPT, input_variables=["context", "question"]
)
chain_type_kwargs = {"prompt": PROMPT}

qa = RetrievalQA.from_chain_type(llm,
            chain_type = "stuff",
            retriever = docssearch.as_retriever(),
            return_source_documents = True,
                                 chain_type_kwargs = chain_type_kwargs)

query = "Which anime would you suggest for someone new to anime?"
result = qa({"query": query})
print(result['result'])

Considering you're new to anime and 18 years old, I'd suggest "Gakuen Alice" as a great starting point. It's a heartwarming comedy with a mix of school life, supernatural powers, and adventure, which are all popular and engaging themes in anime. The show has a lighthearted tone and is easy to follow, making it a great introduction to the world of anime. Additionally, it's a shoujo anime, but it has a broad appeal that can be enjoyed by both males and females, so I think you'll find it entertaining and relatable. Give it a try!


#### Setup for **Gardio UI**

In [ ]:
import gradio as gr
import pandas as pd
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.prompts import PromptTemplate

# Load anime dataset
anime = pd.read_csv("/content/drive/MyDrive/Anime Movie Recommender/anime_with_synopsis.csv")
anime = anime.dropna()
anime['combined_info'] = anime.apply(lambda row: f"Title: {row['Name']}. Overview: {row['sypnopsis']} Genres: {row['Genres']}", axis=1)

# Save updated dataset
anime[['combined_info']].to_csv('updated_anime.csv', index=False)
loader = CSVLoader(file_path="updated_anime.csv")
data = loader.load()

# Text splitting
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(data)

# Create embeddings
embeddings = HuggingFaceEmbeddings()
docssearch = Chroma.from_documents(texts, embeddings)
retriever = docssearch.as_retriever()

# Define LLM
llm = ChatGroq(api_key="your api key", model="llama-3.3-70b-specdec", temperature=0)

# Custom Prompt Template
template = """You are a movie recommender system that helps users find anime that match their preferences.
Use the following pieces of context to answer the question at the end.
For each question, suggest three anime, with a short description of the plot and the reason why the user might like it.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Your response:"""

PROMPT = PromptTemplate(template=template, input_variables=["context", "question"])
chain_type_kwargs = {"prompt": PROMPT}

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs
)

def recommend_anime(query):
    result = qa_chain({'query': query})
    return result['result']

# Gradio interface
theme = "Respair/Shiki@1.2.1"

iface = gr.Interface(
    fn=recommend_anime,
    inputs=gr.Textbox(label="Enter your anime preference"),
    outputs=gr.Textbox(label="Recommended Anime"),
    title="Anime Movie Recommender",
    theme=theme,

)

if __name__ == "__main__":
    iface.launch()


<ipython-input-23-5c7a23a81969>:26: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()


theme_schema%401.2.1.json:   0%|          | 0.00/14.5k [00:00<?, ?B/s]

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0d1395d9da82334cf6.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
